# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 15 2022 2:41PM,246549,20,8526765,"ACI Healthcare USA, Inc.",Released
1,Sep 15 2022 2:34PM,246548,10,MSP214005,"Methapharm, Inc.",Released
2,Sep 15 2022 2:34PM,246548,10,MSP214007,"Methapharm, Inc.",Released
3,Sep 15 2022 2:06PM,246546,10,8530579,Yusen – 3D Matrix,Released
4,Sep 15 2022 1:39PM,246542,10,SNAP915221,"Snap Medical Industries, LLC",Released
5,Sep 15 2022 1:39PM,246542,10,SNAP915222,"Snap Medical Industries, LLC",Released
6,Sep 15 2022 1:39PM,246542,10,SNAP915223,"Snap Medical Industries, LLC",Released
7,Sep 15 2022 12:17PM,246532,10,0085983034,ISDIN Corporation,Released
8,Sep 15 2022 12:17PM,246532,10,0085983035,ISDIN Corporation,Released
9,Sep 15 2022 12:17PM,246532,10,0085983043,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,246532,Released,16
23,246542,Released,3
24,246546,Released,1
25,246548,Released,2
26,246549,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246532,NaN,16.0
246542,NaN,3.0
246546,NaN,1.0
246548,NaN,2.0
246549,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246068,0.0,1.0
246319,1.0,0.0
246324,1.0,0.0
246325,0.0,1.0
246462,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246068,0,1
246319,1,0
246324,1,0
246325,0,1
246462,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246068,0,1
1,246319,1,0
2,246324,1,0
3,246325,0,1
4,246462,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246068,,1
1,246319,1,
2,246324,1,
3,246325,,1
4,246462,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 15 2022 2:41PM,246549,20,"ACI Healthcare USA, Inc."
1,Sep 15 2022 2:34PM,246548,10,"Methapharm, Inc."
3,Sep 15 2022 2:06PM,246546,10,Yusen – 3D Matrix
4,Sep 15 2022 1:39PM,246542,10,"Snap Medical Industries, LLC"
7,Sep 15 2022 12:17PM,246532,10,ISDIN Corporation
23,Sep 15 2022 12:06PM,246529,10,Yusen – 3D Matrix
24,Sep 15 2022 12:06PM,246528,10,"Nextsource Biotechnology, LLC"
26,Sep 15 2022 11:56AM,246526,10,Emerginnova
29,Sep 15 2022 11:55AM,246525,10,Aerogen LTD.
30,Sep 15 2022 11:20AM,246517,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 15 2022 2:41PM,246549,20,"ACI Healthcare USA, Inc.",,1
1,Sep 15 2022 2:34PM,246548,10,"Methapharm, Inc.",,2
2,Sep 15 2022 2:06PM,246546,10,Yusen – 3D Matrix,,1
3,Sep 15 2022 1:39PM,246542,10,"Snap Medical Industries, LLC",,3
4,Sep 15 2022 12:17PM,246532,10,ISDIN Corporation,,16
5,Sep 15 2022 12:06PM,246529,10,Yusen – 3D Matrix,,1
6,Sep 15 2022 12:06PM,246528,10,"Nextsource Biotechnology, LLC",,2
7,Sep 15 2022 11:56AM,246526,10,Emerginnova,,3
8,Sep 15 2022 11:55AM,246525,10,Aerogen LTD.,,1
9,Sep 15 2022 11:20AM,246517,16,Sartorius Bioprocess Solutions,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 15 2022 2:41PM,246549,20,"ACI Healthcare USA, Inc.",1,
1,Sep 15 2022 2:34PM,246548,10,"Methapharm, Inc.",2,
2,Sep 15 2022 2:06PM,246546,10,Yusen – 3D Matrix,1,
3,Sep 15 2022 1:39PM,246542,10,"Snap Medical Industries, LLC",3,
4,Sep 15 2022 12:17PM,246532,10,ISDIN Corporation,16,
5,Sep 15 2022 12:06PM,246529,10,Yusen – 3D Matrix,1,
6,Sep 15 2022 12:06PM,246528,10,"Nextsource Biotechnology, LLC",2,
7,Sep 15 2022 11:56AM,246526,10,Emerginnova,3,
8,Sep 15 2022 11:55AM,246525,10,Aerogen LTD.,1,
9,Sep 15 2022 11:20AM,246517,16,Sartorius Bioprocess Solutions,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 15 2022 2:41PM,246549,20,"ACI Healthcare USA, Inc.",1,
1,Sep 15 2022 2:34PM,246548,10,"Methapharm, Inc.",2,
2,Sep 15 2022 2:06PM,246546,10,Yusen – 3D Matrix,1,
3,Sep 15 2022 1:39PM,246542,10,"Snap Medical Industries, LLC",3,
4,Sep 15 2022 12:17PM,246532,10,ISDIN Corporation,16,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 15 2022 2:41PM,246549,20,"ACI Healthcare USA, Inc.",1.0,NaN
1,Sep 15 2022 2:34PM,246548,10,"Methapharm, Inc.",2.0,NaN
2,Sep 15 2022 2:06PM,246546,10,Yusen – 3D Matrix,1.0,NaN
3,Sep 15 2022 1:39PM,246542,10,"Snap Medical Industries, LLC",3.0,NaN
4,Sep 15 2022 12:17PM,246532,10,ISDIN Corporation,16.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 15 2022 2:41PM,246549,20,"ACI Healthcare USA, Inc.",1.0,0.0
1,Sep 15 2022 2:34PM,246548,10,"Methapharm, Inc.",2.0,0.0
2,Sep 15 2022 2:06PM,246546,10,Yusen – 3D Matrix,1.0,0.0
3,Sep 15 2022 1:39PM,246542,10,"Snap Medical Industries, LLC",3.0,0.0
4,Sep 15 2022 12:17PM,246532,10,ISDIN Corporation,16.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3204714,37.0,8.0
15,493029,6.0,0.0
16,1232434,5.0,0.0
19,246486,1.0,0.0
20,246549,1.0,0.0
21,985036,2.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3204714,37.0,8.0
1,15,493029,6.0,0.0
2,16,1232434,5.0,0.0
3,19,246486,1.0,0.0
4,20,246549,1.0,0.0
5,21,985036,2.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,37.0,8.0
1,15,6.0,0.0
2,16,5.0,0.0
3,19,1.0,0.0
4,20,1.0,0.0
5,21,2.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,37.0
1,15,Released,6.0
2,16,Released,5.0
3,19,Released,1.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Executing,8.0,0.0,0.0,0.0,0.0,2.0
Released,37.0,6.0,5.0,1.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Executing,8.0,0.0,0.0,0.0,0.0,2.0
1,Released,37.0,6.0,5.0,1.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Executing,8.0,0.0,0.0,0.0,0.0,2.0
1,Released,37.0,6.0,5.0,1.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()